Hello! I'm @ceshine. Someone mentioned this open dataset about weekly movie box office numbers in Taiwan, and I've found it very interesting (albeit quite noisy at the same time). I scraped the data from the website and put it on Kaggle. 

This is a preliminary notebook exploring this dataset. We'll quickly browse the dataset, highlight some of the problems of this dataset, and make some visualizations in this notebook.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from datetime import datetime, timedelta

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df = pd.read_csv("/kaggle/input/taiwanweeklyboxoffice/box_office.csv", parse_dates=["week", "release_date"])
df.tail()

## Top Movies in the Latest Week Available

In [ ]:
latest_week = df.week.max()
df_latest = df[df.week == latest_week].copy().sort_values("revenue", ascending=False)
print(f"{df_latest.shape[0]} movies this week")
df_latest.head()

In [ ]:
fig = px.scatter(df_latest.iloc[:30], x="revenue", y="theaters", color="country", hover_data=['name'])
fig.update_layout(
    title_text=f"{latest_week.strftime('%Y%m%d')} Box Office (Top 30)", 
    xaxis_title_text='Revenue ($/TWD in log scale)', # xaxis label
    yaxis_title_text='Theaters', # yaxis label
    width=800,
    height=400,
    xaxis_type="log",
    margin=dict(l=20, r=20, t=50, b=50),
    template="plotly_white"
)
fig.show()

## The Data Collection Issue

The data collector made a decision to only publish numbers of movies that were in theater for the whole 7 days in that week. That means **a lot of the opening week/weekend data (a lot of movies release on Friday or Saturday) will not be in the dataset**. This is a major problem, as a large portion of the tickets sales happens in the opening week.

To demonstrate, let's check out this movie that were released on 2020/07/15. We only have one row of data, but you can see that `tickets != total_tickets` and `revenue != total_revenue`. That's because its sales data from 2020/07/15 to 2020/07/19 did not appear in the weekly data from 2020/07/13 to 2020/07/19.

In [ ]:
df[df.name == "屍速列車：感染半島"]

Here's another example. You can see that this movie sold more tickets in its opening weekend than the next whole week:

In [ ]:
df[df.name == "我的A級秘密"]

"Wait! This is easy to solve. We can get the sales number by subtracting the numbers of the following full week from the total numbers!" You might say at this point. But it's more complicated than it seems.

There are quite a few old move got re-released into the theaters, and the number of this kind of movies are higher during the COVID crisis. Because most of these movies were release before the start of this dataset (Oct 2017), and we have no data on when the movie was re-released, we have no way to recover numbers for the opening week/weekend of the re-release.

Following is an example of this situation. The `total_tickets` and `total_revenue` columns include all sales since 2016/11/25, and we have no way to recover the numbers for the opening week (possibly around 2020/05/01) of this re-release.

In [ ]:
df[df.name == "終極追殺令"]

The above example also highlights another problem of this dataset — **the `release_date` column is inconsistent!** This is really annoying. A way to mitigate this is to use the combination of the name of the movie and the agent/publisher as the unique identifier.

In [ ]:
print(df[["name"]].drop_duplicates().shape[0])
print(df[["name", "agent"]].drop_duplicates().shape[0])
print(df[["name", "publisher"]].drop_duplicates().shape[0])
print(df[["name", "agent", "publisher"]].drop_duplicates().shape[0])

### Recover the opening week data

We can still recover the data if the movie was release after Oct 2017). Let's salvage what we can (BTW, the number of theaters couldn't be recovered so we can only assume it's the same as in the next week):

In [ ]:
df_tmp = df[df.release_date >= "2017-11-01"].sort_values("week").groupby(["name", "publisher"], as_index=False).first()
# No movie was released on Monday
print(df_tmp.shape[0], df_tmp[df_tmp.week == df_tmp.release_date].shape[0])

In [ ]:
df_tmp.head(3)

In [ ]:
tmp = []
for _, row in df_tmp.iterrows():
    row = row.copy()
    row["total_revenue"] = row["total_revenue"] - row["revenue"]
    row["total_tickets"] = row["total_tickets"] - row["tickets"]
    row["revenue"] = row["total_revenue"]
    row["tickets"] = row["total_tickets"]
    assert (row["revenue"] >= 0) and (row["tickets"] >= 0)
    row["week"] = row["week"] - timedelta(days=7)
    tmp.append(row)
df_recovered = pd.DataFrame(tmp)
df_recovered.head(3)

In [ ]:
df = pd.concat([df, df_recovered], axis=0, ignore_index=True).sort_values(["week", "revenue"], ascending=[True, False]).reset_index(drop=True)
df.head(3)

In [ ]:
print("Problematic entries:")
cnt = df.groupby(["name", "publisher", "week"])["week"].count()
print(cnt[cnt > 1])
# throw away a random row for now
print(df.shape[0])
df = df.drop_duplicates(["name", "publisher", "week"])
print(df.shape[0])

In [ ]:
df[df.name == "我的A級秘密"]

In [ ]:
df[df.name == "屍速列車：感染半島"]

## Quartly Gross Revenue

(As explained in the previous section, this data is note 100% accurate because it does not include the opening week of the re-released movies.)

In [ ]:
df["quarter"] = df["week"].apply(lambda x: datetime(x.year, (x.month - 1) // 3 * 3 + 1, 1))
df.quarter.unique()

In [ ]:
df_quarterly = df.groupby("quarter", as_index=False)[["tickets", "revenue"]].sum()

In [ ]:
fig = go.Figure(data=[
    go.Bar(
        x=df_quarterly.quarter.dt.date,
        y=df_quarterly.revenue.values
    )
])
fig.update_layout(
    title_text="Quarterly Box Office Revenue in Taiwan", 
    xaxis_title_text='Quarter', # xaxis label
    yaxis_title_text='Revenue ($/TWD)', # yaxis label
    width=800,
    height=400,
    bargap=0.2, 
    barmode="stack",
    margin=dict(l=20, r=20, t=50, b=50),
    template="plotly_white"
)
fig.show()

## Top Movies in 2019

In [ ]:
df_2019 = df[(df.week >= "2019-01-01") & (df.week < "2020-01-01")]
df_2019_sums = df_2019.groupby(["name", "country"], as_index=False)["tickets", "revenue"].sum().sort_values("revenue", ascending=False)
df_2019_sums

In [ ]:
fig = go.Figure(data=[
    go.Bar(
        x=df_2019_sums.iloc[:20].name.values,
        y=df_2019_sums.iloc[:20].revenue.values
    )
])
fig.update_layout(
    title_text="Top 20 Movies in 2019", 
    xaxis_title_text='Quarter', # xaxis label
    yaxis_title_text='Revenue ($/TWD)', # yaxis label
    width=800,
    height=400,
    bargap=0.2, 
    barmode="stack",
    margin=dict(l=20, r=20, t=50, b=50),
    template="plotly_white"
)
fig.show()

In [ ]:
df_tmp = df_2019_sums.iloc[:20].sort_values("tickets", ascending=False)
fig = go.Figure(data=[
    go.Bar(
        x=df_tmp .name.values,
        y=df_tmp .tickets.values
    )
])
fig.update_layout(
    title_text="Top 20 Movies in 2019", 
    xaxis_title_text='Quarter', # xaxis label
    yaxis_title_text='Tickets Sold', # yaxis label
    width=800,
    height=400,
    bargap=0.2, 
    barmode="stack",
    margin=dict(l=20, r=20, t=50, b=50),
    template="plotly_white"
)
fig.show()

In [ ]:
pd.DataFrame({"Ticket Ranking": df_tmp.name.values, "Revenue Ranking": df_2019_sums.iloc[:20].name.values})

In [ ]:
df_2019[df_2019.name == "驚奇隊長"]